In [ ]:

# connection au WS
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config + target compute ressource
script_config = ScriptRunConfig(source_directory=".",
                                script='training_script.py',
                                compute_target = 'jcharley2',
                                environment=sklearn_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='test4')
run = experiment.submit(config=script_config)
run.wait_for_completion()


In [ ]:
#Récupération des fichiers de modèle
# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

# Create a model folder in the current directory
os.makedirs('./output', exist_ok=True)

# Download the model from run history
run.register_model(model_name='model_save1', model_path='./output',description="Finetuning Similarity model")



TRACKING DES MODELES - LOCAL

In [ ]:
from azureml.core import Model
model = Model.register(workspace=ws,
                       model_name='rf',
                       model_path='output/model.pkl', # local path
                       description='rf test1',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Afficher les modèles inscrits

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

ENREGISTREMENT D'UN JEU DE DONNEES

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/train_dataset.csv')]
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

RUN UN MODELE ENREGISTRE

In [ ]:
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')

SCRIPT DE SCORING / INFERENCE TEMPS REELS
En règle générale, vous utilisez la fonction init pour charger le modèle à partir du registre de modèles et la fonction run pour générer des prédictions à partir des données d’entrée. L’exemple de script suivant illustre ce modèle :

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

In [ ]:
# Créer l'environnement associé
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

In [ ]:
#Une fois toute la configuration préparée, vous pouvez déployer le modèle
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)